# Hardware Crypto Attack

So far we have mostly been talking about software crypto. But how can we expand this to hardware crypto? Luckily it takes very few changes, so you don't have much to do!

In this lab we'll be looking at what is required to attack a hardware crypto device, and what sort of attacks work on these devices. In this case we're going to "cheat" and use an already recorded power trace, since we don't have hardware crypto on our target boards.

## Capture

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
CRYPTO_TARGET = 'HWAES'
N = 3000

In [ ]:
%%bash
cd ../hardware/victims/firmware/
mkdir -p simpleserial-aes-lab1 && cp -r simpleserial-aes/* $_

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes-lab1
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

scope.adc.samples = 2000
scope.gain.mode = "high"
scope.gain.gain = 40
print(scope)

In [ ]:
import time
prog = cw.programmers.STM32FProgrammer
fw_path = "../hardware/victims/firmware/simpleserial-aes-lab1/simpleserial-aes-CW308_STM32F4.hex"
scope.io.target_pwr = False
time.sleep(0.1)
scope.io.target_pwr = True
cw.program_target(scope, prog, fw_path)

Before we do a full (longer) capture, let's set a real-time plot up so we can check the settings. You might want to tweak the gain for example here.

In [ ]:
%run "Helper_Scripts/plot.ipynb"
plot = real_time_plot(plot_len=scope.adc.samples)

In [ ]:
from tqdm import tnrange
ktp = cw.ktp.Basic()
for i in tnrange(20, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    plot.send(trace.wave)   

If all looks OK - go ahead and prepare the full capture. The following will capture N traces, we set that earlier to some large number (like 3000).

In [ ]:
project = cw.create_project("projects/stm32f415.cwp", overwrite = True)

from tqdm import tnrange
ktp = cw.ktp.Basic()
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)
    
project.save()

## Analysis

Next, we'll add our traces to a preprocessing module. We can feed `project.traceManager()` right into `attack.setTraceSource()`, but we could also add pre-processing inbetween (more about this later). We'll also re-open the traces, in this case it is required since the call to `closeAll()` would have flushed the buffers.

In [ ]:
#We also rebuild the project object in case you only want to run this half
project = cw.open_project('projects/stm32f415.cwp')

This time we're going to do a few things. First we will get the traces, and plot a few of them as-is. You can adjust the traces plotted by adjusting the `range(10)`. For example `range(1)` plots the first trace.

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
for i in range(0,10):
    plt.plot(project.waves[i])

If this all works - let's just continue the attack! You'll notice the attack model will change. Let's switch this around for the "last round state" model. Remember though - the "last round state" model will recover the last round key. In order to maintain consistent PGE information, we should also tell the attack we are looking for the last-round key. We do that by adding a `process_known_key` attribute that performs the key scheduling.

In [ ]:
import chipwhisperer.analyzer as cwa
from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds

leak_model = cwa.leakage_models.last_round_state_diff
attack = cwa.cpa(project, leak_model)

def process_key(key):
    recv_key = key_schedule_rounds(key, 0, 10)
    return recv_key

attack.process_known_key = process_key
attack.point_range = [1000, 1500] #Adjust this range as needed! Ideally a single point might be nice.
print(attack)

And then actually run it:

In [ ]:
import chipwhisperer as cw

cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 100)

This might not work exactly - we might need to adjust the window area. So instead we should go back and update the area we specify for the points of interest.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

bnum = 6

key = attack.knownKey()
data = attack.getStatistics().diffs[bnum]
xr = range(0, len(data[0]))

for v in range(0, 256):
    p.line(xr, data[v], line_color='green')

p.line(xr, data[key[bnum]], line_color='red')
show(p)

You should see a graph of red and green in time (samples). In red is the correlation of the correct subkey for the first byte, while the rest are in green. You can use this graph to help fine-tune the windowing of the data. Remember the X-axis is an offset from the previously configured starting point. So if your original window was [1000,1500], and there is a nice peak at 338, your new window should encompass just that peak. So a new window of [1337,1339] might be good to try.

## Analysis with LASCAR

You might have noticed how slow the previous analysis was. While the ChipWhisperer core wasn't designed for performance, but we can luckily use some other platforms to run the analysis with. In this example we'll use LASCAR. You may need to install it (the VM release has this already, but if on your own computer you'll need to install). Doing so is as simple as:

    git clone https://github.com/Ledger-Donjon/lascar.git
    cd lascar
    #Can run something like this if you want to keep ref to sources
    python setup.py develop
    
You might need to restart the kernel afterwards to pick up the new module. You can again use the `start_point` and `end_point` to fine-tune the attack.

In [ ]:
import chipwhisperer as cw
import numpy as np

#project = cw.openProject("../stm32f415_lab.cwp")

project = cw.open_project("./projects/stm32f415.cwp")

start_point = 1000
end_point = 1500

tm = project.trace_manager()

trace_array = np.zeros( (tm.num_traces(), end_point - start_point))
textin_array = np.zeros( (tm.num_traces(), len(tm.get_textin(0))), dtype="uint8" )
textout_array = np.zeros( (tm.num_traces(), len(tm.get_textout(0))), dtype="uint8" )

print ("Copying %d traces of %d samples into memory" % (tm.num_traces(), tm.num_points()))
for n in range(0, tm.num_traces()):
    trace_array[n] = tm.get_trace(n)[start_point:end_point]
    textin_array[n] = tm.get_textin(n)
    textout_array[n] = tm.get_textout(n)
print("Copying done! Thank you.")

In [ ]:
from lascar import *
from lascar.tools.aes import sbox, inv_sbox

#The following leakage models copied from /chipwhisperer/analyzer/attacks/models/AES128_8bit.py and
# massaged into Lascar Version

def selection_function_lastroundHD(byte):
    # selection_with_guess function must take 2 arguments: value and guess
    def selection_with_guess(value, guess):
        INVSHIFT_undo = [0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12, 1, 6, 11]
        st10 = value[INVSHIFT_undo[byte]]
        st9 = inv_sbox[value[byte] ^ guess]
        return hamming(st9 ^ st10)
    return selection_with_guess

In [ ]:
from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds

#Example - round 10 (final round key), for state-to-state leakage
highlight_key = key_schedule_rounds(project.trace_manager().get_known_key(0), 0, 10)

#Adjust this for actual attack used!
guess_range = range(256)
cpa_engines = [CpaEngine("cpa_%02d" % i, selection_function_lastroundHD(i), guess_range) for i in range(16)]

#Leakage models using ciphertext (such as lastroundHD) require textout
containter_textout =  TraceBatchContainer(trace_array, textout_array) #<--textin_array to textout
session = Session(containter_textout, engines=cpa_engines).run(batch_size=50)

In [ ]:
import pandas as pd
from IPython.display import clear_output
import numpy as np
import chipwhisperer as cw
from chipwhisperer.analyzer.attacks._stats import Results

class LascarCWAttacks(object):   
    def __init__(self, cpa_engines, highlight_key=None):  
        dt = Results()
        for i in range(len(cpa_engines)):
            results = cpa_engines[i].finalize()
            dt.update_subkey(i, results)

        self.dt = dt
        self.hlk = highlight_key
        
    def format_stat(self, stat):
        return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

    def color_corr_key(self, row):
        ret = [""] * 16
        for i,bnum in enumerate(row):
            if bnum[0] == highlight_key[i]:
                ret[i] = "color: red"
            else:
                ret[i] = ""
        return ret
    
    def show_pge(self):
        stat_data = self.dt.find_maximums()
        df = pd.DataFrame(stat_data).transpose()
        return df.head().style.format(self.format_stat).apply(self.color_corr_key, axis=1)
        
    def results(self): #getStatistics
        """CW Interfae Function"""
        return self.dt
    
    def known_key(self): #knownKey
        """CW Interface Function"""
        if self.hlk is None: return [0]*16
        
        return self.hlk
    
    def getReportingInterval(self):
        """CW Interface Function"""
        return 0
    
results = LascarCWAttacks(cpa_engines, highlight_key)
results.show_pge()

In [ ]:

from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

key = highlight_key

data = results.results().diffs[0]
xr = range(0, len(data[0]))

for v in range(0, 256):
    p.line(xr, data[v], line_color='green')

for bnum in range(0, 16):
    data = results.results().diffs[bnum]
    p.line(xr, data[key[bnum]], line_color='red')
show(p)


## Conclusion

Attacking hardware crypto is similar to any other DPA style attack. In this example we have concentrated on the standard "Last Round State" to break a real hardware accelerator.